In [8]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob

from sklearn import datasets
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

%matplotlib notebook

In [11]:
data_dir = "-6db_fan/id_00/normal"
normal_audio_files = glob(data_dir + '/*.wav')
x = len(normal_audio_files)
x

0

In [6]:
data_dir = "-6dB_fan/fan/id_00/abnormal"
abnormal_audio_files = glob(data_dir + '/*.wav')
y = len(abnormal_audio_files)
y


0

In [5]:
extracted_features_df=pd.read_csv('csv_files/-6db/fan_id_00_normal.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'csv_files/-6db/fan_id_00_normal.csv'

In [ ]:
extracted_features_df

In [69]:
y_specs = []

In [72]:
for file in tqdm(normal_audio_files):
    file_name = file
    data, sr = librosa.load(file_name)
    y_min = data.min()
    y_max = data.max()
    y_mean = data.mean()
    y_std = data.std()
    
    y_specs.append([y_min,y_max,y_mean,y_std])

100%|██████████████████████████████████████████████████████████████████████████████| 1011/1011 [03:15<00:00,  5.17it/s]


In [73]:
for file in tqdm(abnormal_audio_files):
    file_name = file
    data, sr = librosa.load(file_name)
    y_min = data.min()
    y_max = data.max()
    y_mean = data.mean()
    y_std = data.std()
    
    y_specs.append([y_min,y_max,y_mean,y_std])

100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [01:18<00:00,  5.20it/s]


In [98]:
y_specs = np.array(y_specs)

In [101]:
y_cols =['y_min','y_max','y_mean','y_std']
for i,name in enumerate(y_cols):
    print(i,name)
    extracted_features_df[name]=y_specs[:,i]

extracted_features_df
    
    

0 y_min
1 y_max
2 y_mean
3 y_std


,file_name,zcr,spec_band_sum,spec_band_max,spec_band_min,spec_band_mean,spec_band_std,rmse_mean,class,y_min,y_max,y_mean,y_std
0,-6dB_fan/fan/id_00/normal\00000000.wav,0.033,553328.314402,2186.859945,1199.106924,1283.824395,56.193976,0.010253,0,-0.042073,0.047081,-7.359320e-06,0.010293
1,-6dB_fan/fan/id_00/normal\00000001.wav,0.074,754206.782847,1964.498781,1601.458971,1749.899728,46.057723,0.009026,0,-0.036983,0.035768,-6.922498e-07,0.009064
2,-6dB_fan/fan/id_00/normal\00000002.wav,0.044,610831.291691,1611.333345,1322.125918,1417.241976,31.888845,0.010177,0,-0.041308,0.043374,-1.956126e-06,0.010220
3,-6dB_fan/fan/id_00/normal\00000003.wav,0.090,768171.338788,1903.834562,1646.409253,1782.300090,47.540587,0.008910,0,-0.036632,0.036621,-2.262496e-06,0.008968
4,-6dB_fan/fan/id_00/normal\00000004.wav,0.026,601052.624223,1859.185695,1245.459978,1394.553652,89.329761,0.010531,0,-0.052299,0.051868,-2.138482e-06,0.010785
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,-6dB_fan/fan/id_00/abnormal\00000402.wav,0.060,589684.631613,1991.972897,1016.851359,1368.177800,188.335089,0.009734,1,-0.080353,0.079435,-2.909642e-06,0.010613
1414,-6dB_fan/fan/id_00/abnormal\00000403.wav,0.070,661583.479851,2016.924958,1288.197948,1534.996473,116.475586,0.010159,1,-0.049392,0.041839,-6.262130e-06,0.010208
1415,-6dB_fan/fan/id_00/abnormal\00000404.wav,0.038,580425.358225,2010.045068,1247.780494,1346.694567,46.210524,0.010522,1,-0.042902,0.047687,6.785936e-07,0.010568
1416,-6dB_fan/fan/id_00/abnormal\00000405.wav,0.029,568262.217837,1917.085187,1162.319349,1318.473823,122.268276,0.010673,1,-0.051811,0.045138,-8.994813e-06,0.010715


In [135]:
extracted_features_df.to_csv('csv_files/-6db/fan_id_00_normal.csv',index=False)

In [106]:
def filter_features(df,y,treshold=0.10):
    relevant_features_indx = []
    for i in range(len(df.columns)):
        print(f"{df.columns[i]}: {np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]}, threshold:{treshold}, boolean:{abs(np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]) >treshold}")
        if abs(np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]) >treshold:
            relevant_features_indx.append(i)
    return relevant_features_indx

f_cols = filter_features(extracted_features_df.drop(["file_name","class"],axis=1).iloc[:,:-1],extracted_features_df.loc[:,'class'].values,treshold=0.10)


zcr: -0.10930470303878594, threshold:0.1, boolean:True
spec_band_sum: -0.06542224939287412, threshold:0.1, boolean:False
spec_band_max: -0.011468991200941388, threshold:0.1, boolean:False
spec_band_min: -0.07141624528573425, threshold:0.1, boolean:False
spec_band_mean: -0.06542224939287421, threshold:0.1, boolean:False
spec_band_std: 0.02054083793113262, threshold:0.1, boolean:False
rmse_mean: 0.43549674057780924, threshold:0.1, boolean:True
y_min: -0.20560460854601845, threshold:0.1, boolean:True
y_max: 0.16266834484743958, threshold:0.1, boolean:True
y_mean: -0.007552479433436432, threshold:0.1, boolean:False


In [146]:
X_train, X_test, y_train, y_test = train_test_split(extracted_features_df.loc[:,['rmse_mean']].values, extracted_features_df.loc[:,'class'].values, test_size=0.3, random_state=42)

In [147]:
pipeline_lr = Pipeline([('scalar1',StandardScaler()),
                        ('lr_classifier',LogisticRegression(random_state=0))])

pipeline_rf = Pipeline([('scalar2',StandardScaler()),
                        ('rf_classifier',RandomForestClassifier(n_estimators=int((len(extracted_features_df.columns)**0.5//1)), criterion= 'entropy',random_state=0))])

pipeline_dt = Pipeline([('scalar3',StandardScaler()),
                        ('dt_classifier',DecisionTreeClassifier(criterion="entropy",random_state=0))])

pipeline_GaussianNB = Pipeline([('scalar4',StandardScaler()),
                        ('dt_classifier',GaussianNB())])


In [148]:
pipelines = [pipeline_lr,pipeline_rf,pipeline_dt,pipeline_GaussianNB]

pipe_dict = {0:'Logistic Regression',1:'Random Forest',2:'Decision Tree',3:'GaussianNB'}

In [149]:
for pipe in pipelines:
    pipe.fit(X_train,y_train)

In [150]:
best_accuracy=0.0
best_classifier_indx = 0
best_pipeline=""

In [151]:
for i,model in enumerate(pipelines):
    print(f"{pipe_dict[i]} test accuracy: {model.score(X_test,y_test)} ")
    if model.score(X_test,y_test)>best_accuracy:
        best_accuracy=  model.score(X_test,y_test)
        best_pipeline=model
        best_classifier_idnx = i
print(f"Classifier with the best accuracy:{pipe_dict[best_classifier_idnx]}")

Logistic Regression test accuracy: 0.8286384976525821 
Random Forest test accuracy: 0.7417840375586855 
Decision Tree test accuracy: 0.7089201877934272 
GaussianNB test accuracy: 0.8286384976525821 
Classifier with the best accuracy:Logistic Regression


In [137]:
data_dir = "0dB_fan/fan/id_00/normal"
normal_audio_files = glob(data_dir + '/*.wav')

data_dir = "0dB_fan/fan/id_00/abnormal"
abnormal_audio_files = glob(data_dir + '/*.wav')

from tqdm import tqdm
extracted_feature = []

for file in tqdm(normal_audio_files):
    file_name = file
    class_label = 0 # 0 for normal.
    data, sr = librosa.load(file_name)
   

    rms = rmse(data)
    rms_mean = rms.mean()
        
   

    extracted_feature.append([file_name,rms_mean,class])

  0%|                                                                                         | 0/1011 [00:00<?, ?it/s]


NameError: name 'zero_crossing_rate' is not defined

In [ ]:
for file in tqdm(abnorm_audio_files):
    file_name = file
    class_label = 1 # 1 for abnormal.
    data, sr = librosa.load(file_name)
    y_min = data.min()
    y_max = data.max()
    y_mean = data.mean()
    y_std = data.std()
    
    zcr = zero_crossing_rate(data)
    zcr_mean = zcr.mean()
    zcr_max = zcr.max()
    zcr_min = zcr.min()
    
    spec_cent = spec_centroid(data,sr)
    spec_cent_min = spec_cent.min()
    spec_cent_mean = spec_cent.mean()

    spec_roll = spec_rolloff(data, sr)
    spec_roll_min = spec_roll.min()  
    spec_roll_mean = spec_roll.mean() 

    rms = rmse(data)
    rms_mean = rms.mean()
        
    flat = flatness(data)
    flat_mean = flat.mean()
    flat_max = flat.max()
        
    amplitude = amplitude_envelope(data)
    amp_mean = amplitude.mean()
    amp_min = amplitude.min()
        
    mel_spec = mel_spectrogram(data,sr)
    melspec_mean = mel_spec.mean()
    melspec_std = mel_spec.std()

    extracted_feature.append([file_name, y_min, y_max, y_mean, y_std, zcr_mean, zcr_min, zcr_max,
                              spec_cent_min, spec_cent_mean, spec_roll_min, spec_roll_mean, rms_mean,
                              flat_mean, flat_max, amp_mean, amp_min, melspec_mean, melspec_std, class_label])

In [157]:
extracted_features_df2=pd.read_csv('csv_files/0db/id_00_normal.csv')

In [158]:
def filter_features(df,y,treshold=0.10):
    relevant_features_indx = []
    for i in range(len(df.columns)):
        print(f"{df.columns[i]}: {np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]}, threshold:{treshold}, boolean:{abs(np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]) >treshold}")
        if abs(np.corrcoef(df.values[:,i],np.reshape(y,-1))[1][0]) >treshold:
            relevant_features_indx.append(i)
    return relevant_features_indx

f_cols = filter_features(extracted_features_df2.drop(["file_name","class"],axis=1).iloc[:,:-1],extracted_features_df2.loc[:,'class'].values,treshold=0.10)


In [159]:
y_pred = pipelines[0].predict(extracted_features_df.loc[:,['rmse_mean']].values)

In [160]:
pd.DataFrame([y_pred,extracted_features_df2['class']])

,0,1,2,3,4,5,6,7,8,9,...,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
